<a href="https://colab.research.google.com/github/freshpex/NLP-Assignment/blob/main/NLP_Assignment_Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data


In [ ]:
# import these modules
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))

# a denotes adjective in "pos"
print("better :", lemmatizer.lemmatize("corpora rocks cats", pos="a"))


[nltk_data] Downloading package wordnet to /root/nltk_data...


rocks : rock
corpora : corpus
better : corpora rocks cats


In [ ]:
# This dataset contains ~25k tweets in English.
!wget -O mini_twitter.txt https://raw.githubusercontent.com/mhassanist/nlp_text_datasets/ea689be7f27906d0e27b112bc11311d40967b5a3/mini_twitter.txt

--2024-06-03 21:38:53--  https://raw.githubusercontent.com/mhassanist/nlp_text_datasets/ea689be7f27906d0e27b112bc11311d40967b5a3/mini_twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1747021 (1.7M) [text/plain]
Saving to: ‘mini_twitter.txt’

mini_twitter.txt    100%[===================>]   1.67M  --.-KB/s    in 0.05s   

2024-06-03 21:38:54 (36.7 MB/s) - ‘mini_twitter.txt’ saved [1747021/1747021]



In [ ]:
# Open the file and loads its content
text = ''
with open('mini_twitter.txt', 'r') as content_file:
    text = content_file.read()

In [ ]:
# Make sure your data is loaded correctly.
# Task 1: Print the first 100 character of the text

# YOUR CODE HERE
print(text[:100])


How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way 


# Import required libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
import nltk

# Tokenization

In [ ]:
# Task 2.1: Convert the data to lowercase and split it into separate sentences. Store the result in the corpus variable
corpus = text.lower().split("\n")

print(corpus[0]) # how are you? btw thanks for the rt. you gonna be in dc anytime soon? love to see you. been way, way too long.


how are you? btw thanks for the rt. you gonna be in dc anytime soon? love to see you. been way, way too long.


In [ ]:
# Task 2.2: Convert all sentences from the text corpus into n-gram sequences of tokens (numeric representations of words)

from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize a tokenizer
tokenizer = Tokenizer()

# Fit it on the text
tokenizer.fit_on_texts(corpus)


In [ ]:
# Task 2.3 : # Use the tokenizer to convert each sentence to n-gram sequence
input_sequences = []

# YOUR CODE HERE
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[1]) # [54, 24, 5]



[56, 22, 5]


# LSTM Model

In [ ]:
# Task 3: Create LSTM model, compile it and fit it.

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 10, input_length=max_sequence_len - 1))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=20, verbose=1)




# Model Prediction

In [ ]:
# Complete the following prediction function

# Prediction function
def predict_next_word(text):
    predicted_word = ''


    # YOUR CODE HERE

    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    predicted_word = tokenizer.index_word[predicted_word_index]


    return predicted_word


# test the model
print(predict_next_word("most people mess up")) # many

print(predict_next_word("most people mess up many")) # many times


# Play with it !

In [ ]:
text = "let's" # starting text
for _ in range(10): # generate the next 10 words
  next = predict_next_word(text)
  text = text + " " + next

print(text)
